# SC-ATACSEQ-EXPLORER PIPELINE

This single cell ATAC-Seq analysis pipeline is designed for intergative analysis of
dataset, initially processed by 10x Genomics.

**In addition** to 10x Genomics results it offers:

* Flexible and clear data preprocessing and normalization methods (including UMI and per peak length)
* Summary on different conditions in case of aggregated dataset
* Noise and duplets filtering during cell calling
* Different types of clustering followed by heatmap and UMAP / t-SNE visualizations in low dimensions space
* Top cluster markers visualization on t-SNE plot
* Closest genes annotations for peaks and clusters
* Annotated cell-specific genes analysis
* BigWig and BED files for clusters and markers ready-to-be-visualized in [JBR Genome Browser](https://research.jetbrains.org/groups/biolabs/tools/jbr-genome-browser) by JetBrains Research
* Data preparation for single cell explorer by [Artyomov Lab](http://artyomovlab.wustl.edu/site/), Washington University in St.Louis

Required 10x Genomics Cell Cell Ranger ATAC files:

* `fragments.tsv` - fragments matrix file provided by Cell Ranger
* `peaks.bed` - peaks file (union of peaks) in case of merging
* `clusters.csv` - Graph clustering after median normalization, IDF scaling, SVD projection and L2 normalization
* `projection.csv` - T-SNE 2d project of all the cells

<hr>

Other pipelines:

* **Cell Ranger ATAC-Seq** https://support.10xgenomics.com/single-cell-atac/software/pipelines/latest/algorithms/overview
* **Seurat** https://www.biorxiv.org/content/biorxiv/early/2018/11/02/460147.full.pdf (not for sc-ATAC-Seq https://github.com/satijalab/seurat/issues/1422)
* **Scasat** https://academic.oup.com/nar/article/47/2/e10/5134327
* **SnapATAC** https://www.biorxiv.org/content/10.1101/615179v2
* **Scater** https://academic.oup.com/bioinformatics/article/33/8/1179/2907823

In [ ]:
# Configuration
FRAGMENTS_FILE = '/mnt/stripe/shpynov/2019_gxfer1_DT1634_Denis/BAAJ_Spleen/outs/fragments.tsv.gz'
# PEAKS_FILE = '/mnt/stripe/shpynov/2019_gxfer1_DT1634_Denis/union_peaks.bed'
PEAKS_FILE = '/mnt/stripe/shpynov/2019_gxfer1_DT1634_Denis/merged_200_1.0E-10_2.peak.0E-10_2.peak'
CLUSTERS_FILE = '/mnt/stripe/shpynov/2019_gxfer1_DT1634_Denis/BAAJ_Spleen/outs/analysis/clustering/graphclust/clusters.csv'
TSNE_FILE = '/mnt/stripe/shpynov/2019_gxfer1_DT1634_Denis/BAAJ_Spleen/outs/analysis/tsne/2_components/projection.csv'

# ! wget http://mitra.stanford.edu/kundaje/akundaje/release/blacklists/mm10-mouse/mm10.blacklist.bed.gz
# ! guznip mm10.blacklist.bed.gz
BLACKLIST_FILE = '/mnt/stripe/shpynov/2019_gxfer1_DT1634_Denis/mm10.blacklist.bed'
# ! wget ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M22/gencode.vM22.annotation.gtf.gz 
# ! gunzip gencode.vM22.annotation.gtf.gz
GTF_FILE = '/mnt/stripe/shpynov/2019_gxfer1_DT1634_Denis/gencode.vM22.annotation.gtf'

# representative DNase hypersensitivity sites for mm10 https://www.encodeproject.org/annotations/ENCSR489XTV/
DNASE_FILE = '/mnt/stripe/shpynov/2019_gxfer1_DT1634_Denis/ENCFF108APF.bed'

FACTOR = 'age'
FACTORS_MAP = {1: 'old', 2: 'young'}
FACTOR_FUNCTION = lambda x: 1 if x.endswith('-1') else 2

OUTPUT_DIR = '/mnt/stripe/shpynov/2019_gxfer1_DT1634_Denis/pipeline'

In [ ]:
MARKERS_REGIONS = {
    'GZMK classic promoter': 'chr13:113180223-113181928',
    'GZMK alternative promoter': 'chr13:113182148-113184892',
    'CD4 promoter': 'chr6:124887932-124888480',
    'CD8a promoter': 'chr6:71373114-71373837',
    'CD3d promoter': 'chr9:44981140-44982928',
    'CD19 promoter': 'chr7:126414090-126415383',
    'NCR1 promoter': 'chr7:4337172-4338250',
    'CCl5 promoter': 'chr11:83530163-83530832'
}

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import glob
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import re
import seaborn as sns
sns.set_style("whitegrid")

from pybedtools import BedTool
from tqdm.auto import tqdm
from scipy.stats import zscore

# Fix random seed
np.random.seed(0)

# Peaks file stats

In [ ]:
peaks_df = pd.read_csv(PEAKS_FILE, sep='\t', header=None)
print('Peaks', len(peaks_df))
sns.distplot(peaks_df[2] - peaks_df[1])
plt.title('Peak length distribution')
plt.show()

# Overlap of peaks with representative DNAse hypersensitive sites

In [ ]:
dnase = BedTool(DNASE_FILE)
peaks_file = BedTool(PEAKS_FILE)
peak_count = peaks_file.count()
overlap = peaks_file.intersect(dnase, wa=True, u=True).count()
peak_by_dnase = 0 if overlap == 0 else overlap * 100.0 / peak_count
overlap = dnase.intersect(peaks_file, wa=True, u=True).count()
dnase_by_peak = overlap * 100.0 / dnase.count()
print('Fraction of peaks overlapping with representative DNAse', int(peak_by_dnase), '%')
print('Fraction of representative DNAse overlapping with peaks', int(dnase_by_peak), '%')

# Code for GTF file preprocessing

In [ ]:
print('Preprocess GTF file')

gtf_file = GTF_FILE
transcripts_bed = gtf_file + '.bed'
if not os.path.exists(transcripts_bed):
    GTF_TO_BED_SH = """
TNF=$(cat $1 | grep transcript_id | grep -v '#' | head -n 1 | awk '{for (i=4; i<NF; i++) {if ($3=="transcript" && $i=="gene_name") print (i+1)}}')
cat $1 | awk -v TNF=$TNF -v OFS="\t" '{if ($3=="transcript") {print $1,$4-1,$5,$TNF,0,$7}}' | tr -d '";' | sort -k1,1 -k2,2n -k3,3n --unique > $2
    """
    with tempfile.NamedTemporaryFile(prefix='gtf_to_bed', suffix='.sh', delete=False) as f:
        f.write(GTF_TO_BED_SH.encode('utf-8'))
        f.close()
        ! bash {f.name} {gtf_file} {transcripts_bed} 
print('Transcripts bed file', transcripts_bed)

transcripts_tss = gtf_file + '.tss'
if not os.path.exists(transcripts_tss):
    BED_TO_TSS_SH = """
cat $1 | awk -v OFS="\t" '{if ($6=="+") {print $1, $2, $2+1, $4, 0, "+"} else {print $1,$3-1,$3, $4, 0, "-"}}' | sort -k1,1 -k2,2n -k3,3n --unique > $2
    """
    with tempfile.NamedTemporaryFile(prefix='bed_to_tss', suffix='.sh', delete=False) as f:
        f.write(BED_TO_TSS_SH.encode('utf-8'))
        f.close()
        ! bash {f.name} {transcripts_bed} {transcripts_tss} 
print('Transcripts tss file', transcripts_tss)

# Code for clusters summary

In [ ]:
def clusters_summary(df, value):
    df_sum = df[['cluster', value]].groupby(['cluster']).sum().reset_index()
    df_sum_age = df[['cluster', FACTOR, value]].groupby(['cluster', FACTOR]).sum().reset_index()

    plt.figure(figsize=(10, 4))
    cmap=plt.cm.get_cmap('jet', len(df_sum))
    sns.barplot(x=df_sum.index, y=df_sum[value],
                palette=[cmap(i) for i in range(len(df_sum))])

    plt.show()

    print('Proportions')
    plt.figure(figsize=(10, 4))
    cmap=plt.cm.get_cmap('jet', len(FACTORS_MAP))
    
    # Plot 1 - background - "total" 
    sns.barplot(x = df_sum.index, y = np.ones(len(df_sum)) * 100, color = cmap(0), alpha=0.5)
    
    percentages = np.zeros(len(df_sum))
    for i, (k,v) in enumerate(FACTORS_MAP.items()):
        if i > 0:
            percentages += np.array(df_sum_age.loc[df_sum_age[FACTOR] == k][value]) / np.array(df_sum[value]) * 100.0
            # Plot N - overlay - "bottom" series
            bottom_plot = sns.barplot(x=df_sum.index, y=percentages, color = cmap(i), alpha=0.5)


    legend_colors = [plt.Rectangle((0,0),1,1, fc=cmap(i), edgecolor = 'none') for i in range(len(FACTORS_MAP))]
    legend_names = [v for _,v in FACTORS_MAP.items()]
    l = plt.legend(legend_colors, legend_names, prop={'size':16})
    l.draw_frame(False)

    #Optional code - Make plot look nicer
    sns.despine(left=True)
    bottom_plot.set_ylabel('Cluster composition ' + FACTOR + ' %')
    bottom_plot.set_xlabel('Cluster')

    plt.show()

# Code for scatterplot visualization

In [ ]:
def plot_colored(data, c, name='component', n_clusters=None, show_centers=False, alpha=0.5):
    """ Plot colored scatterplot """
    cmap = plt.cm.get_cmap('jet', n_clusters) if n_clusters else plt.cm.viridis
    if data.shape[1] >= 3:
        fig, ax = plt.subplots(1, figsize=(12, 10))
        plt.subplots(nrows=1, ncols=3, figsize=(18, 5))
        for i, (c1,c2) in enumerate([(0, 1), (0, 2), (1, 2)]):
            plt.setp(ax, xticks=[], yticks=[])
            plt.subplot(1, 3, i + 1)
            plt.scatter(data[:, c1], data[:, c2], 
                        c=c, 
                        cmap=cmap, 
                        marker='o',
                        edgecolor='none', alpha=alpha)
            plt.xlabel('{} {}'.format(name, c1 + 1))
            plt.ylabel('{} {}'.format(name, c2 + 1))
            plt.colorbar()
    else:
        fig, ax = plt.subplots(1, figsize=(12, 10))
        sc = plt.scatter(data[:,0], data[:,1], 
                         c=c, cmap=cmap,
                         marker='o',
                         edgecolor='none', alpha=alpha)
        plt.setp(ax, xticks=[], yticks=[])
        if n_clusters and show_centers:
            # For each cluster, we add a text with name
            for cluster in range(n_clusters):
                cdata = data[[i for i in range(len(c)) if int(c[i]) == cluster], :]
                cluster_mean = cdata.mean(axis=0)
                plt.text(cluster_mean[0], cluster_mean[1], str(cluster), 
                         horizontalalignment='center', size='large', color='black', weight='bold')            
        if n_clusters:
            cbar = plt.colorbar(boundaries=np.arange(n_clusters + 1) + min(c) - 0.5)
            cbar.set_ticks(np.arange(n_clusters) + min(c))
            cbar.set_ticklabels(list(set(c)))
        else:
            plt.colorbar()
    plt.show()

# 10x Genomics Cell Ranger

This section provides additional analysis and visualization of Cell Ranger ATAC output.

In [ ]:
clusters10xpath = CLUSTERS_FILE
clusters10xdf = pd.read_csv(clusters10xpath, sep=',')
clusters10xdf['Cluster'] = clusters10xdf['Cluster'].astype(int) - 1 # Clusters start with 1 in 10x
clusters10xdf[FACTOR] = [FACTOR_FUNCTION(bc) for bc in clusters10xdf['Barcode']]
display(clusters10xdf.head())
print('Total clusters', len(set(clusters10xdf['Cluster'])))

In [ ]:
print('Size of clusters analysis')
t = clusters10xdf.copy()
t['counts'] = np.ones(len(t))
t.rename(columns={'Cluster': 'cluster'}, inplace=True)
t.index = t['Barcode']
print('Summary by', FACTOR, FACTORS_MAP)
display(t[[FACTOR, 'counts']].groupby([FACTOR]).sum())

clusters_summary(t[['cluster', 'counts', FACTOR]], 'counts')

In [ ]:
print('Cell Ranger ATAC original T-SNE visualization')
tsne10xpath = TSNE_FILE
tsne10xdf = pd.read_csv(tsne10xpath, sep=',')

mergeddf = pd.merge(left=tsne10xdf,
                    right=clusters10xdf, 
                    left_on='Barcode', right_on='Barcode')

plot_colored(mergeddf[['TSNE-1', 'TSNE-2']].values, mergeddf['Cluster'], 
             name='tsne', 
             n_clusters=len(set(mergeddf['Cluster'])),
             show_centers=True)

In [ ]:
print('tSNE on LSA normalized vs', FACTOR, FACTORS_MAP)
plot_colored(mergeddf[['TSNE-1', 'TSNE-2']].values, clusters10xdf[FACTOR], 'tsne', 2, alpha=0.2)

# Cell Calling

**Pipeline** uses **noise_threshold** and **duplets_threshold**.

<hr>

**Cell Ranger ATAC** aggregation: when combining data from multiple GEM groups, the cellranger-atac aggr pipeline automatically equalizes the sensitivity of the groups before merging, which is the recommended approach in order to avoid the batch effect introduced by sequencing depth. **Default method**: Subsample fragments from higher-depth GEM wells until they all have an equal number of unique fragments per cell.

For each barcode:
* we have the record of mapped high-quality fragments that passed all filters (the fragments.tsv file).
* Having determined peaks prior to this, we use the number of fragments that overlap any peak regions.
* Separate the signal from noise.

**This works better in practice as compared to naively using the number of fragments per barcode.** 
We first subtract a depth-dependent fixed count from all barcode counts to model whitelist contamination. This fixed count is the estimated number of fragments per barcode that originated from a different GEM, assuming a contamination rate of 0.02. Then we fit a mixture model of two negative binomial distributions to capture the signal and noise. Setting an odds ratio of 1000, we separate the barcodes that correspond to real cells from the non-cell barcodes.

**Scasat** If a cell has open peaks below a user defined threshold (default: 50 peaks) in the peak accessibility matrix we would remove that cell from subsequent analysis. Also peaks not observed across a user defined number of valid cells (default: 10 cells) are not considered for further downstream analysis.

**SnapATAC** alternative approach -  1kb, 5kb and 10kb resolution.

Existing computational methods rely on **pre-defined** regions of transposase accessibility
identified from the aggregate signals. ...

**Limitations**: 
1. It requires sufficient number of single cell profiles to create robust aggregate signal for peak calling. 
2. The cell type identification is biased toward the most abundant cell types in the tissues. 
3. These techniques lack the ability to reveal regulatory elements in the rare cell populations which are underrepresented in the aggregate signal. This concern is critical, for example, in brain tissue, where key neuron types may represent less than 1% of all cells while still playing a critical role in the neural circuit.


In [ ]:
import re

# Fragments file provided by cell ranger
fragments_file_gz = FRAGMENTS_FILE

# BedTools doesn't work with gz file, so unzip it
!gunzip {fragments_file_gz}
fragments_file = re.sub('\.gz', '', fragments_file_gz)

In [ ]:
# Intersect using bedtools rather than pybedtools, because they are too slow for files of this size!
import tempfile
import pandas as pd
import numpy as np
import os

def intersect_fragments_and_peaks(fragments_file, peaks_file, blacklist_file):
    print('Fragments')
    ! wc -l {fragments_file}
    idf = None
    print('Blacklist')
    ! wc -l {blacklist_file}
    print('Peaks')
    ! wc -l {peaks_file}

    with tempfile.TemporaryDirectory(prefix='pipeline') as td:
        print('Filtering out non-standard chromosomes')
        chr_filtered_file = os.path.join(td, 'chromosome_filtered.bed')
        ! cat {fragments_file} | grep -i -E 'chr[0-9mt]+[^_]' > {chr_filtered_file}
        ! wc -l {chr_filtered_file}

        print('Blacklist regions filtration')
        blacklist_filtered_file = os.path.join(td, 'blacklist_filtered.bed')
        ! bedtools intersect -v -a {chr_filtered_file} -b {blacklist_file} > {blacklist_filtered_file}
        ! wc -l {blacklist_filtered_file}

        print('Fragments and peaks intersection')
        intersection_file = os.path.join(td, 'intersection.bed')
        ! bedtools intersect -wa -wb -a {blacklist_filtered_file} -b {peaks_file} > {intersection_file}
        ! wc -l {intersection_file}

        idf = pd.read_csv(intersection_file, sep='\t', header=None)
        icolnames = ['chr', 'start', 'end', 'barcode', 'reads', 'peak_chr', 'peak_start', 'peak_end']
        assert len(idf.columns) >= len(icolnames)
        idf.rename(columns={o: n for (o, n) in zip(idf.columns[:len(icolnames)], icolnames)}, inplace=True)
        idf = idf[icolnames] # Reorder and drop others
    print('Done intersecting of fragments and peaks')
    return idf

idf = intersect_fragments_and_peaks(fragments_file, PEAKS_FILE, BLACKLIST_FILE)

In [ ]:
def filter_cells(idf, noise_threshold=None, duplets_threshold=None):
    # 10x Genomics Cell Ranger ATAC-Seq marks duplicates
    # 'count' ignores multiple reads per barcode at same position
    pidf = pd.pivot_table(idf, values='reads', index=['barcode'], aggfunc='count')
    pidf.reset_index(level=0, inplace=True)
    
    print('Total barcodes', len(pidf))    
    plt.figure(figsize=(8, 8))
    # n is for shoulder graph
    counts = sorted(pidf['reads'], reverse=True)
    df_counts = pd.DataFrame(data={'count': counts, 'n': range(1, len(counts) + 1)})
    cells_filter = [True] * len(df_counts)
    
    if noise_threshold is not None:
        noise_filter = df_counts['count'] <= noise_threshold
        cells_filter = np.logical_and(cells_filter, np.logical_not(noise_filter))
        df_noise = df_counts.loc[noise_filter]
        print('Noise', len(df_noise))
        plt.plot(np.log10(df_noise['n']), 
                 np.log10(df_noise['count']), 
                 label='Noise', linewidth=3, color='red')        

    if duplets_threshold is not None:
        duplets_filter = df_counts['count'] >= duplets_threshold
        cells_filter = np.logical_and(cells_filter, np.logical_not(duplets_filter))
        df_duplets = df_counts.loc[duplets_filter]
        print('Duplets', len(df_duplets))
        plt.plot(np.log10(df_duplets['n']), 
                 np.log10(df_duplets['count']), 
                 label='Duplets', linewidth=3, color='orange')        

    df_cells = df_counts.loc[cells_filter]

    cells_filter = [True] * len(pidf)
    if noise_threshold is not None:
        cells_filter = np.logical_and(cells_filter, pidf['reads'] > noise_threshold)
    if duplets_threshold is not None:
        cells_filter = np.logical_and(cells_filter, pidf['reads'] < duplets_threshold)
        
    cells = set(pidf.loc[cells_filter]['barcode'])
    idfcells = idf.loc[[c in cells for c in idf['barcode']]]
    print('Estimated number of cells', len(cells))

    print('Preparing UMI shoulder graph')
    plt.plot(np.log10(df_cells['n']), 
             np.log10(df_cells['count']), 
             label='Cell', linewidth=3, color='green')

    plt.xlabel('Log10 number of barcodes')
    plt.ylabel('Log10 fragments overlapping Peaks')
    plt.legend(loc='upper right')
    plt.show()

    print('Cells UMI distribution')
    sns.distplot(np.log10(df_cells['count']))
    plt.title('UMI summary coverage distribution')
    plt.xlabel('Log10 summary coverage')
    plt.ylabel('Frequency')
    plt.show()

    return idfcells

In [ ]:
idfcells = filter_cells(idf, noise_threshold=200, duplets_threshold=8000)

## Peak-Barcode Matrix

**Pipeline**
Compute number of different UMIs which intersect with peak. Multiple copies of UMI are ignored.

<hr>

**Cell Ranger ATAC-Seq**
We produce a count matrix consisting of the counts of fragment ends (or cut sites) within each peak region for each barcode. This is the raw peak-barcode matrix and it captures the enrichment of open chromatin per barcode. The matrix is then filtered to consist of only cell barcodes, which is then used in subsequent analysis such as dimensionality reduction, clustering and visualization.

A barcoded fragment may get sequenced multiple times due to PCR amplification. We mark duplicates in order to identify the original fragments that constitute the library and contribute to its complexity.

In [ ]:
print('Barcode vs summary fragments overlap with peaks')
# 10x Genomics Cell Ranger ATAC-Seq marks duplicates
# 'count' ignores multiple reads per barcode at same position
pdf = pd.pivot_table(idfcells, values='reads', 
                     index=['peak_chr', 'peak_start', 'peak_end', 'barcode'], 
                     aggfunc='count').reset_index()

pdf['peak'] = pdf['peak_chr'] + ':' + pdf['peak_start'].astype(str) + '-' + pdf['peak_end'].astype(str)
pdf.drop(columns=['peak_chr', 'peak_start', 'peak_end'], inplace=True)
# display(pdf.head())

In [ ]:
print('Transforming dataframe to peaks x barcode format')
fulldf = pd.pivot_table(pdf, index='peak', columns='barcode', values='reads').fillna(0)
# Remove extra labels from pivot_table columns
fulldf.columns = fulldf.columns.values
fulldf.index.name = None
# display(fulldf.head())

In [ ]:
print('Summary UMI distribution per factor')
factors = [FACTOR_FUNCTION(bc) for bc in fulldf.columns]
for k, v in FACTORS_MAP.items():
    print('Processing', v)
    sns.kdeplot(np.log10(fulldf[[c for i, c in enumerate(fulldf.columns) if factors[i] == k]].sum()), label=v)

plt.title('UMI summary coverage distribution')
plt.xlabel('Summary coverage')
plt.ylabel('Frequency')
plt.legend()
plt.show()

# Normalization

**Pipeline** 
* Global scaling to median coverage of UMI for coverage reporting
* Peak length normalization for cluster analysis

<hr>

**Cell Ranger** uses normalization to median coverage depth in each UMI - we do the same here.

**Seurat** we employ a global-scaling normalization method “LogNormalize” that normalizes the feature expression measurements for each cell by the total expression, multiplies this by a scale factor (10,000 by default), and log-transforms the result.

**SnapATAC** does not require population-level peak annotation, and instead
assembles chromatin landscapes by directly clustering cells based on the similarity of
their genome-wide accessibility profile. Using a regression-based normalization
procedure, SnapATAC adjusts for differing read depth between cells.

In [ ]:
from diffexp import estimate_size_factors

print('UMI normalization by median fragments count per barcode (RPM)') 
# (peaks x barcodes) format
size_factors = estimate_size_factors(fulldf.values)
normdf = fulldf / size_factors # per column, i.e. barcode

In [ ]:
print('Peak length normalization')
lengths = [int(re.split(':|-', p)[2]) - int(re.split(':|-', p)[1]) for p in fulldf.index]
rpkdf = fulldf.divide(lengths, axis=0) * 1000

# Feature selection (Identification of highly variable features)

**Pipeline**

* mean >99% as alignment errors or housekeeping genes
* std <1% as noise

<hr>

**Cell Ranger ATAC** removed features selection with zero variance.

**SnapATAC**
The vast majority of the items in the cell-by-bin count matrix is “0”, some items have abnormally high coverage (often > 200) perhaps due to alignment error. Bins of exceedingly high coverage which likely represent the genomic regions that are invariable between cells such as housekeeping gene promoters were removed. We noticed that filtering bins of extremely low coverage perhaps due to random noise can also improve the robustness of the downstream clustering analysis. 
* Calculated the coverage of each bin using the binary matrix and 
* Removed the top **0.1%** items of the highest coverage
* Normalized the coverage by log10(count + 1). 
* Log-scaled coverage obey approximately a gaussian distribution which is then converted into zscore. 
* Bins with zscore beyond ±2 were filtered before further analysis.

**Seurat** calculate a subset of features that exhibit high cell-to-cell variation in the dataset (i.e, they are highly expressed in some cells, and lowly expressed in others). We and others have found that focusing on these genes in downstream analysis helps to highlight biological signal in single-cell datasets.

Seurat feature selection visualization
<img src='assets/4.png' width=20%>

In [ ]:
def feature_selection(df, stdp=1, meanp=99):
    # df is (peaks x barcodes)
    print('Computing Mean vs SD of coverage')
    means = df.T.mean()
    stds = df.T.std()
    
    plt.figure(figsize=(8, 8))
    sns.jointplot(x=means, y=stds)
    plt.title('Mean vs SD')
    plt.show()

    # Filter out most highly covered peaks as alignment errors or housekeeping genes. 
    means_high = np.percentile(means, meanp)
    peaks_filter = means < means_high
    
    # Filter out non-variable peaks
    stds_low = np.percentile(stds, stdp)
    peaks_filter = np.logical_and(stds_low < stds, means < means_high)
    peaks_filter_indexes = np.flatnonzero(peaks_filter)
            
    print('After feature selection')
    plt.figure(figsize=(8, 8))
    sns.jointplot(x=means[peaks_filter_indexes], y=stds[peaks_filter_indexes])
    plt.title('Mean vs SD')
    plt.show()

    return peaks_filter

In [ ]:
# Normalization to peak length to take into account signal strength, not total coverage
peaks_filter = feature_selection(rpkdf, meanp=99)
print('Total peaks', len(rpkdf))
print('Filtered peaks', sum(peaks_filter))

In [ ]:
print('Check marker regions of interest')
peaks = fulldf.index[np.flatnonzero(peaks_filter)]
marker_regions_peaks = {}
for k, r in MARKERS_REGIONS.items():
    v = re.split(':|-', r)
    tp = [(c,s,e) for (c,s,e) in map(lambda p: re.split(':|-', p), peaks) 
        if c==v[0] and int(s) > int(v[1]) and int(e) < int(v[2])]
    assert tp, f'{k} not found'
    assert len(tp) == 1, f'Too many peaks for region {k}'
    marker_regions_peaks[k] = c,s,e = tp[0]
    print('Region', k, 'Peak', f'{c}:{s}-{e}')

# Dimensionality Reduction, Clustering and t-SNE Projection

**Pipeline** Use Cell Ranger ATAC approach and IRLB code for dimensionality reduction.

<hr>

**Cell Ranger ATAC** default method is **LSA**.

1. IDF scaling
2. IRLBA SVD decomposition to dim 15
3. L2 normalization in low dimension projection
4. Graph clustering
5. T-SNE visualization

The number of dimensions is fixed to 15 as it was found to sufficiently separate clusters visually and in a biologically meaningful way when tested on peripheral blood mononuclear cells (PBMCs).
We found that the combination of these normalization techniques obviates the need to remove the first component.

## PCA (median and log scaling + classic algorithm)
1. UMI normalization
2. Log scale
3. IRLBA SVD decomposition to dim 10 instead of PCA

In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler

# print('PCA on features peaks vs log10 sum coverage')
# # Number of dimensions recommended by Cell Ranger ATAC-Seq algorithm
# pca = PCA(n_components=15)
# result_pca = pca.fit_transform(np.log1p(normdf.loc[peaks_filter].T)) # (n_samples x n_features)
# print('Explained variation', np.sum(pca.explained_variance_ratio_))
# plot_colored(result_pca, np.log10(normdf.sum()), 'pca')

## LSA (IDF + IRLBA + L2 normalization)

In [ ]:
from sklearn.preprocessing import Normalizer
from irlb import irlb

def lsa(df, dim):
    print('Normalizing by IDF')
    idf = np.log1p(df.shape[1]) - np.log1p(np.count_nonzero(df, axis=1))
    df_idf = df.multiply(idf, axis=0).T # Transpose to (barcodes x peaks) format
    
    print('Performing IRLBA without scaling or centering')
    # Number of dimensions recommended by Cell Ranger ATAC-Seq algorithm
    (_, _, v, _, _) = irlb(df_idf, dim)
    # project the matrix to complete the transform: X --> X*v = u*d
    df_idf_irlb = df_idf.dot(v) 
    assert df_idf_irlb.shape == (df_idf.shape[0], dim), 'check dimensions'

    # IMPORTANT! We found that the combination of these normalization techniques 
    # obviates the need to remove the first component.
    print('Normalization each barcode data point to unit L2-norm in the lower dimensional space')
    idf_irlb_l2 = Normalizer(norm='l2').fit_transform(df_idf_irlb) # (n_samples, n_features) 
    return pd.DataFrame(idf_irlb_l2, index=df_idf_irlb.index, columns=df_idf_irlb.columns)

# NO UMI normalization required for LSA approach!
# IDF is invariant to scales, IRLBA don't need scaling.
# Normalization to peak length to take into account signal strength, not total coverage
result_lsa = lsa(rpkdf.loc[peaks_filter], 15)

In [ ]:
# UMAP visualization
import umap
umap_values = umap.UMAP().fit_transform(result_lsa.values)
print('UMAP on LSA normalized vs log10 coverate depth')
plot_colored(umap_values, np.log10(fulldf.sum()), 'umap')

## Graph clustering of LSA normalized data

Clustering should:
* produce clusters enough clusters 10-15 recommended by Cell Ranger
* too small clusters are likely artifacts < 100 cells

In [ ]:
from collections import Counter
from sklearn.cluster import AgglomerativeClustering

def graph_clustering(result, n_clusters):
    # Ward method as default
    print('Clustering', n_clusters)
    clusters = AgglomerativeClustering(n_clusters=n_clusters).fit_predict(result).astype(int)
    cluster_counter = Counter()
    for c in clusters:
        cluster_counter[c] += 1

    # reorder clusters by size descending
    clusters_reord = np.zeros(len(clusters), dtype=int)    
    for i, (c, n) in enumerate(cluster_counter.most_common()):
        clusters_reord[clusters == c] = i

    return clusters_reord

# 10-15 clusters is recommened values by Cell Ranger ATAC-Seq
n_clusters = 15
clusters = graph_clustering(result_lsa, n_clusters)
print('Saving clusters to clusters{}.tsv'.format(n_clusters))
clusters_df = pd.DataFrame({'cluster': clusters}, index=result_lsa.index)
clusters_df.to_csv(OUTPUT_DIR + '/clusters{}.tsv'.format(n_clusters))

In [ ]:
n = 100
print('Sampling', n, 'cells from cluster')
t = result_lsa # (barcodes x lsa coordinates) format
cluster_dfs = []
for cluster in tqdm(range(n_clusters)):
    cluster_df = t.loc[clusters == cluster].sample(n=n, axis=0, replace=True)
    cluster_df['cluster'] = cluster/n_clusters # Normalize cluster number to 0-1
    cluster_dfs.append(cluster_df)
t = pd.concat(cluster_dfs)

print('QC: clustermap of normalized LSA coordinates and clusters')
sns.clustermap(t, col_cluster=False, row_cluster=False, figsize=(8, 8))
plt.show()

In [ ]:
print('Computing summary for clusters')
t = fulldf.T.copy()
t[FACTOR] = [FACTOR_FUNCTION(bc) for bc in t.index]
t['cluster'] = clusters
t['counts'] = np.ones(len(t))

print('Summary by', FACTOR, FACTORS_MAP)
display(t[[FACTOR, 'counts']].groupby([FACTOR]).sum())

clusters_summary(t, 'counts')

In [ ]:
print('Summary UMI distribution per cluster')
plt.figure(figsize=(15, 8))
for c in tqdm(range(n_clusters)):
    sns.kdeplot(np.log10(fulldf.T.loc[[clusters[i] == c for i in range(len(clusters))]].sum(axis=1)), 
                label=str(c))

plt.title('UMI summary coverage distribution')
plt.xlabel('Summary coverage')
plt.ylabel('Frequency')
plt.legend()
plt.show()

# UMAP for dimensionality reduction and visualization

In [ ]:
print('UMAP on LSA normalized vs graph clusters')
plot_colored(umap_values, clusters, 'umap', n_clusters, show_centers=True)

In [ ]:
print('tSNE on LSA normalized vs', FACTOR, FACTORS_MAP)
factors = [FACTOR_FUNCTION(bc) for bc in result_lsa.index]
plot_colored(umap_values, factors, 'tsne', 2, alpha=0.1)

## T-SNE

In [ ]:
from sklearn.manifold import TSNE
print('Processing t-SNE on L2-normalized data')
results_lsa_tsne = pd.DataFrame(TSNE(n_components=2).fit_transform(result_lsa.values),
                                index=result_lsa.index,
                                columns=['tsne1', 'tsne2'])
print('t-SNE done!')
# display(results_lsa_tsne.head())

print('Saving tSNE coordinates to tsne.tsv')
results_lsa_tsne.to_csv(OUTPUT_DIR + '/tsne.tsv', sep='\t')

In [ ]:
print('tSNE on LSA normalized vs log10 coverate depth')
plot_colored(results_lsa_tsne.values, np.log10(fulldf.sum()), 'tsne')

In [ ]:
print('tSNE on LSA normalized vs', FACTOR, FACTORS_MAP)
factors = [FACTOR_FUNCTION(bc) for bc in result_lsa.index]
plot_colored(results_lsa_tsne.values, factors, 'tsne', 2, alpha=0.2)

In [ ]:
print('tSNE on LSA normalized vs graph clusters')
plot_colored(results_lsa_tsne.values, clusters, 'tsne', n_clusters, show_centers=True)

# Marker regions analysis

In [ ]:
def plot_z(xs, ys, zs, ax, title, xlabel, ylabel, alpha=0.5, size=10):
    """Plot zscores scatter plot"""
    plt.setp(ax, xticks=[], yticks=[])
    zscores_pos = zs > 0
    zscores_pos_ind = np.flatnonzero(zscores_pos)
    zscores_neg_ind = np.flatnonzero(np.logical_not(zscores_pos))

    pos_colors = np.zeros((len(zscores_pos_ind), 4))
    pos_colors[:, 0] = 1.0 # red
    pos_colors[:, 3] = alpha * zscores[zscores_pos_ind] / np.max(zscores) # alpha

    neg_colors = np.zeros((len(zscores_neg_ind), 4))
    neg_colors[:, 2] = 1.0 # blue
    neg_colors[:, 3] = alpha * zscores[zscores_neg_ind] / np.min(zscores) # alpha

    ax.scatter(xs[zscores_neg_ind], ys[zscores_neg_ind], edgecolor='none', c=neg_colors, s=size)
    ax.scatter(xs[zscores_pos_ind], ys[zscores_pos_ind], edgecolor='none', c=pos_colors, s=size)
    
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

In [ ]:
print('Z-scores for marker regions on interest')
peaks = normdf.index[np.flatnonzero(peaks_filter)]

n_marker_regions_peaks = len(marker_regions_peaks)
nrows = max([c for c in range(1, math.ceil(math.sqrt(n_marker_regions_peaks))) if n_marker_regions_peaks % c == 0])
ncols = int(n_marker_regions_peaks / nrows)

plt.subplots(nrows=nrows, ncols=ncols, figsize=(4 * ncols, 4 * nrows))
for i, (k, r) in tqdm(enumerate(marker_regions_peaks.items())):
    c,s,e = r
    peak = f'{c}:{s}-{e}'    
    # Z scale for each marker peak across all cells
    zscores = zscore(normdf.loc[peak].T)
    t1 = results_lsa_tsne['tsne1']
    t2 = results_lsa_tsne['tsne2']
    ax = plt.subplot(nrows, ncols, i + 1)
    plot_z(t1, t2, zscores, ax, title=k, xlabel='tSNE axis 1', ylabel='tSNE axis 2', alpha=1)
plt.show()

## Comparison vs 10x Cell Ranger clustering

In [ ]:
print('Our clusters into Cell Ranger ATAC coordinates') 
mergeddf = pd.merge(pd.DataFrame({'Barcode': results_lsa_tsne.index, 'Cluster': clusters}),
                    right=tsne10xdf, 
                    left_on='Barcode', right_on='Barcode')

print('tSNE on LSA normalized vs Cell Ranger ATAC clusters')
plot_colored(mergeddf[['TSNE-1', 'TSNE-2']].values, mergeddf['Cluster'], 
             name='tsne', 
             n_clusters=len(set(mergeddf['Cluster'])),
             show_centers=True)

In [ ]:
print('Clusters from Cell Ranger ATAC into our coordinates') 
mergeddf = pd.merge(pd.DataFrame({'Barcode': results_lsa_tsne.index,
                                 'tsne1': results_lsa_tsne['tsne1'],
                                 'tsne2': results_lsa_tsne['tsne2']}),
                    right=clusters10xdf, 
                    left_on='Barcode', right_on='Barcode')

print('tSNE on LSA normalized vs Cell Ranger ATAC clusters')
plot_colored(mergeddf[['tsne1', 'tsne2']].values, mergeddf['Cluster'], 
             name='tsne', 
             n_clusters=len(set(mergeddf['Cluster'])),
             show_centers=True)

# BigWig RPM normalized profiles

**Pipeline** Split UMI reads by clusters and use RPKM normalization.

<hr>

**Seurat** We created pseudo-bulk ATAC-seq profiles by pooling together cells with for each cell type. Each cell type showed enriched accessibility near canonical marker genes. Chromatin accessibility tracks are normalized to sequencing depth (RPKM normalization) in each pooled group.

**SnapATAC** Next we aggregate cells from the each cluster to create an ensemble track for peak calling and visualization. 

In [ ]:
print('Loading full fragments dataframe')
fdf = pd.read_csv(fragments_file, sep='\t', names=['chr', 'start', 'end', 'barcode', 'reads'])

print('Aggregating fragments and clusters')
df_for_bigwigs = pd.merge(left=fdf, right=clusters_df,
                          left_on='barcode', right_on=clusters_df.index)
display(df_for_bigwigs.head()) # already sorted by chromosomes

## Writing BigWig files

In [ ]:
import glob
import pyBigWig

step = 100
bigwigs_path = OUTPUT_DIR + '/bigwig'
! mkdir -p {bigwigs_path}
    
for cluster in range(n_clusters):
    bw_path = bigwigs_path + '/cluster_{}.bw'.format(cluster)
    if os.path.exists(bw_path):
        print(f'Bigwig file {bw_path} exists, skipping...')
        continue
    print('Processing cluster', cluster, bw_path)
    df_for_bigwigs_cluster = df_for_bigwigs.loc[df_for_bigwigs['cluster'] == cluster]
    chr_lengths = df_for_bigwigs_cluster[['chr', 'end']].groupby('chr').max().reset_index()

    with pyBigWig.open(bw_path, 'w') as bw:
        bw.addHeader(list(zip(chr_lengths['chr'], chr_lengths['end'])))
        for chromosome in tqdm(chr_lengths['chr']):
            df_for_bigwigs_cluster_chr =\
                df_for_bigwigs_cluster.loc[df_for_bigwigs_cluster['chr'] == chromosome].sort_values(['start', 'end'])

            starts = list(df_for_bigwigs_cluster_chr['start'])
            ends = list(df_for_bigwigs_cluster_chr['end'])
            reads = list(df_for_bigwigs_cluster_chr['reads'])
            chr_length = int(chr_lengths.loc[chr_lengths['chr'] == chromosome]['end'])
            values = np.zeros(int(math.floor(chr_length / step)) + 1)

            for i in range(len(df_for_bigwigs_cluster_chr)):
                # Ignore PCR duplicates here!
                # v = reads[i] 
                v = 1.0
                si = int(math.floor(starts[i] / step)) 
                ei = int(math.floor(ends[i] / step)) 
                if ei == si:
                    values[si] += v
                else:
                    values[si] += v / 2 
                    values[ei] += v / 2
            
            non_zero = values > 0
            non_zero_inds = np.flatnonzero(non_zero)
            starts_np = non_zero_inds * step
            ends_np = (non_zero_inds + 1) * step
            values_np = values[non_zero]
            # RPM normalization
            values_np = values_np * (1000000.0 / sum(values_np))
            chroms_np = np.array([chromosome] * sum(non_zero))
            bw.addEntries(chroms_np, starts=starts_np, ends=ends_np, values=values_np)                     

print('Done')

# Closest genes annotation

**Pipeline** Preprocess GENES to TSS and use `bedtools closest -D b` to report distance to TSS, upstream or downstream w.r.t. the gene strand.

<hr>

**Cell Ranger ATAC** We use `bedtools closest -D b` to associate each peak with genes based on closest transcription start sites (packaged within the reference) such that the peak is within 1000 bases upstream or 100 bases downstream of the TSS.

In [ ]:
def locations_closest_genes(locations, transcripts_tss):
    print('Annotate list of locations in format "chr:start-end" with closest tss of genes')
    with tempfile.NamedTemporaryFile(delete=False) as f:
        # Save index to restore original order after sorting
        for i, t in enumerate([re.split(':|-', p) for p in locations]):
            f.write('{}\t{}\t{}\t{}\n'.format(*t, i).encode('utf-8'))
        f.close()
        closest_file = f.name + '.closest'
        sorted_file = f.name + '.sorted'
        ! sort -k1,1 -k2,2n {f.name} > {sorted_file}
        ! bedtools closest -a {sorted_file} -b {transcripts_tss} -D b > {closest_file}
        closest_df = pd.read_csv(
            closest_file, sep='\t', 
            names=['chr', 'start', 'end', 'index', 
                   'gene_chr', 'gene_start', 'gene_end', 'gene', 'gene_score', 'gene_strand', 'distance'])
        # Restore original sorting as 
        closest_df.sort_values(by=['index'], inplace=True)
        # Pick only first closest gene in case of several
        closest_df.drop_duplicates(['index'], inplace=True)
        return closest_df[['chr', 'start', 'end', 'gene', 'distance']]


closest_genes = locations_closest_genes(list(normdf.index), transcripts_tss)
# Restore 'peak' to be able to merge on it
closest_genes['peak'] = closest_genes['chr'] +\
                        ':' + closest_genes['start'].astype(str) +\
                        '-' + closest_genes['end'].astype(str)
display(closest_genes.head())

# Save cluster mean values

In [ ]:
# Transpose to (barcodes x peaks) format
t = normdf.T 

# Per cluster mean values for peaks
clusters_means = {str(c): t.loc[clusters == c].mean() for c in range(n_clusters)}
clusters_means_df = pd.DataFrame(clusters_means).reset_index()[[str(c) for c in range(n_clusters)]]
clusters_means_genes_df = pd.concat([closest_genes.reset_index(), clusters_means_df], axis=1).drop(columns=['index'])
# Save peak without dashes or colons
clusters_means_genes_df['peak'] = [re.sub('[:-]', '_', p) for p in result['peak']]

print('Saving clusters peaks_means to clusters_peaks_values.tsv')
display(clusters_means_genes_df.head())
clusters_means_genes_df.to_csv(OUTPUT_DIR + '/clusters_peaks_values.tsv', sep='\t', index=False)

In [ ]:
print('Hierarchical clustering of clusters mean values')
sns.clustermap(clusters_means_genes_df[[str(i) for i in range(n_clusters)]], 
               col_cluster=True, row_cluster=False, figsize=(10, 10), cmap=plt.cm.seismic)
plt.show()

# Differential peaks (Markers)

**Pipeline**

Use Cell Ranger ATAC to get differential markers. 

<hr>

**Cell Ranger ATAC**
For each peak we perform test cluster vs others, so that several clusters may yield significant results vs others, we take into account only the cluster with greatest mean value.\
Test is is perfromed using Cell Ranger default testing procedure based on Negative Binomial GLM.

In [ ]:
# Launch 10x Genomics Cell Ranger DE
from diffexp import *

def run_10x_differential_expression(df, clusters, fdr=0.05):
    """ Compute differential expression for each cluster vs all other cells
        Args: df          - feature expression data (peak x barcode)
              clusters    - 0-based cluster labels
              fdr         - false discovery rate"""

    peaks = list(df.index)
    distinct_clusters = set(clusters)
    print('Detected clusters', len(distinct_clusters))

    print("Computing params...")
    m = df.values
    sseq_params = compute_sseq_params(m)
    
    cluster_markers = []
    for cluster in distinct_clusters:
        in_cluster = clusters == cluster
        group_a = np.flatnonzero(in_cluster)
        group_b = np.flatnonzero(np.logical_not(in_cluster))
        print('Computing DE for cluster {}...'.format(cluster))

        de_result = sseq_differential_expression(m, group_a, group_b, sseq_params)
        de_result['cluster'] = [cluster] * len(de_result)
        de_result['peak'] = peaks
        passed_fdr = de_result['adjusted_p_value'] < fdr
        passed_log2_fc = de_result['log2_fold_change'] > 0
        # Filter out different by statistical tests with log2 fold change > 0
        passed = np.logical_and(passed_fdr, passed_log2_fc)
        print('DE:', sum(passed), 'of', len(de_result), 
              'FDR:', sum(passed_fdr), 'log2fc>0:', sum(passed_log2_fc))

        passed_de = de_result.loc[passed]
        cluster_de_markers = passed_de[
            ['cluster', 'peak', 'norm_mean_a', 'norm_mean_b', 'log2_fold_change', 'p_value', 'adjusted_p_value']
        ].rename(columns={'norm_mean_a': 'norm_mean_cluster', 'norm_mean_b': 'norm_mean_others'}) 

        cluster_markers.append(cluster_de_markers)    

    return pd.concat(cluster_markers)

In [ ]:
# Cell Ranger ATAC differential analysis
# Each peak is tested independently, no normalization to peak length required.
markers = run_10x_differential_expression(fulldf.loc[peaks_filter], clusters)
print('Total markers', len(markers))
display(markers.head())

# Markers analysis
* Summary
* Top markers visualization on heatmap / T-SNE

In [ ]:
print('Number of differential markers per cluster')
t = markers[['cluster']].copy()
t['count'] = 1
display(pd.pivot_table(t, values='count', index=['cluster'], aggfunc=np.sum))

# Annotate markers with genes

In [ ]:
markers_with_genes = pd.merge(left=markers[['cluster', 'norm_mean_cluster', 'norm_mean_cluster', 
                                            'log2_fold_change', 'p_value', 'adjusted_p_value', 'peak']],
                         right=closest_genes,
                         left_on='peak',
                         right_on='peak')

# Save peak without dashes or colons
markers_with_genes['peak'] = [re.sub('[:-]', '_', p) for p in markers_with_genes['peak']]

# rearrange columns, sort
markers_with_genes = markers_with_genes[['chr', 'start', 'end', 'peak',
                                         'cluster', 'norm_mean_cluster', 'norm_mean_cluster', 
                                         'log2_fold_change', 'p_value', 'adjusted_p_value', 
                                         'gene', 'distance']].sort_values(by=['cluster', 'adjusted_p_value'])
display(markers_with_genes.head())

print('Saving all the markers', len(markers_with_genes), 'to markers.tsv')
markers_with_genes.to_csv(OUTPUT_DIR+ '/markers.tsv', sep='\t', index=None)

# Analyzing top markers

In [ ]:
def get_top_markers(df, n_clusters=n_clusters, top=100):
    top_markers = pd.concat([
        df.loc[df['cluster'] == c].sort_values(by=['adjusted_p_value']).head(top) 
                    for c in range(0, n_clusters)])
    top_markers['peak'] = top_markers['chr'] +\
                          ':' + top_markers['start'].astype(str) + '-'\
                          + top_markers['end'].astype(str)
    return top_markers


top_markers = get_top_markers(markers_with_genes, top=50)
display(get_top_markers(markers_with_genes, top=1))

In [ ]:
print('Computing Z scores for mean values in cluster for markers')
t = normdf.T # Transpose to (barcodes x peaks) format
t = t[list(top_markers['peak'])] # top markers only

t = pd.concat([pd.DataFrame(t.loc[clusters == cluster].mean()).T for cluster in range(n_clusters)])
t.index = range(n_clusters)

# Transform to Z-score
for c in t.columns:
    t[c] = zscore(t[c])

print('Clustermap of Z scores')
sns.clustermap(t.T, col_cluster=False, row_cluster=False, figsize=(10, 10), cmap=plt.cm.seismic)
plt.show()

In [ ]:
print('t-SNE based Z-SCORE visualizations for markers, mean z-score for each cell is given')
nrows = max([c for c in range(1, math.ceil(math.sqrt(n_clusters))) if n_clusters % c == 0])
ncols = int(n_clusters / nrows)

t = normdf

plt.subplots(nrows=nrows, ncols=ncols, figsize=(4 * ncols, 4 * nrows))
for cluster in tqdm(range(n_clusters)):
    # Table (marker for cluster x cells)
    markers_df = t.loc[top_markers.loc[top_markers['cluster'] == cluster]['peak']].T.copy()

    # Z scale for each marker peak across all cells
    for c in markers_df.columns:
        markers_df[c] = zscore(markers_df[c])
    
    # Average Z score for each cell
    zscores = markers_df.T.mean()
    t1 = results_lsa_tsne['tsne1']
    t2 = results_lsa_tsne['tsne2']
    ax = plt.subplot(nrows, ncols, cluster + 1)
    plot_z(t1, t2, zscores, ax, title=f'Cluster {cluster}', xlabel='tSNE axis 1', ylabel='tSNE axis 2')        
#     plt.setp(ax, xticks=[], yticks=[])    
#     sc = ax.scatter(t1, t2, s=5, c=zscores, cmap=plt.cm.seismic, edgecolor='none', alpha=0.5)
#     plt.colorbar(sc)
#     plt.title(f'Cluster {cluster}')
#     plt.xlabel('tSNE axis 1')
#     plt.ylabel('tSNE axis 2')
plt.show()

# Differential markers to BED


In [ ]:
cluster_peaks_path = os.path.join(OUTPUT_DIR, 'markers')
! mkdir -p {cluster_peaks_path}
print('Cleanup peaks_clusters', cluster_peaks_path)
for f in glob.glob('{}/markers_*.bed'.format(cluster_peaks_path)):
    os.remove(f)

mlap_max = (-np.log10(markers.loc[markers['adjusted_p_value'] != 0]['adjusted_p_value'])).max()
print('BED scoring as -log10 adjusted pval')
print('Max of -log10 adjusted pval', mlap_max)

for c in range(n_clusters):
    bed_file = '{}/markers_{}.bed'.format(cluster_peaks_path, c)
    markers_cluster = markers.loc[markers['cluster'] == c].sort_values(
        ['log2_fold_change'], ascending=False).iloc[::-1]
    markers_cluster.index = range(len(markers_cluster))
    print('Writing cluster', c, 'differential peaks to', bed_file)
    with open(bed_file, 'w') as bed:
        for i in range(len(markers_cluster)):
            peak = markers_cluster['peak'][i]
            ap = markers_cluster['adjusted_p_value'][i]
            mlap = -np.log10(ap) if ap != 0.0 else mlap_max
            line = '{}\t{}\t{}\t.\n'.format(
                re.sub(':|-', '\t', peak), 
                c,
                mlap)
            bed.write(line)
print('Done')                

# TOP markers and associated genes
You can copy and paste any of these lists directly to JBR Genome Browser.

In [ ]:
print('Top markers genes with biggest log fold change')
top_markers = get_top_markers(markers_with_genes, top=10)

for cluster in range(n_clusters):
    print('Cluster', cluster)
    cluster_top = top_markers.loc[top_markers['cluster'] == cluster]
    print(', '.join(set(cluster_top['gene'])))
    print('---')
    print(', '.join(cluster_top['peak']))
    print('\n')

# Preparation data for single cell explorer
Preprocess data for single cell explorer http://artyomovlab.wustl.edu/shiny/single_cell_explorer

Upload `data_for_plot.tsv`, `expData.Rda` and `markers.tsv` to corresponding single cell explorer folder and voila!

In [ ]:
from scipy import io, sparse
sce_path = os.path.join(OUTPUT_DIR, 'sce')
! mkdir -p {sce_path}

print('Save data_for_plot.tsv')
barcodes = normdf.T.columns
data_for_plot = pd.DataFrame(
    {'tSNE_1': results_lsa_tsne['tsne1'], 
     'tSNE_2': results_lsa_tsne['tsne2'], 
     'Cluster': clusters,
     'nCounts': fulldf.sum()})
data_for_plot.index = barcodes
data_for_plot.to_csv(os.path.join(sce_path, 'data_for_plot.tsv'), sep='\t')

print('Save RPM expData.Rda.mtx')
csc = sparse.csc_matrix(normdf.values)
io.mmwrite(os.path.join(sce_path, 'expData.Rda.mtx'), csc)

print('Save barcodes.txt')
with open(os.path.join(sce_path, 'barcodes.txt'), 'w') as f:
    for b in normdf.columns.values:
        f.write(b + '\n')

print('Save peaks.txt')
with open(os.path.join(sce_path, 'peaks.txt'), 'w') as f:
    for p in normdf.index.values:
        # Save peak without dashes or colons        
        f.write(re.sub('[:-]', '_', p) + '\n')

print('Save markers.tsv')
markers_with_genes.to_csv(os.path.join(sce_path, 'markers.tsv'), index=None, sep='\t')
        
print('Done')

In [ ]:
print('Converting to Rda format in R')

R_CODE = """
require(readr)
require(Matrix)
library(magrittr)

barcodes <- readr::read_tsv('{0}/barcodes.txt', col_names = F)
peaks <- readr::read_tsv('{0}/peaks.txt', col_names = F)
expData <- Matrix::readMM('{0}/expData.Rda.mtx') %>%
    magrittr::set_rownames(peaks$X1) %>%
    magrittr::set_colnames(barcodes$X1)
save(expData, file = "{0}/expData.Rda")
""".format(sce_path)

with tempfile.NamedTemporaryFile(prefix='prepare', suffix='.R', delete=False) as f:
    f.write(R_CODE.encode('utf-8'))
    f.close()
    ! Rscript {f.name}
print('Cleanup')
os.remove('{}/barcodes.txt'.format(sce_path))
os.remove('{}/peaks.txt'.format(sce_path))
os.remove('{}/expData.Rda.mtx'.format(sce_path))
print('Done')
